# Sobol SA

In [ ]:
#### %matplotlib inline
from model import *
from agents import *
from globals import *
# from server import *
from schedule import *
from utility import *
from SALib.sample import saltelli
from mesa.batchrunner import BatchRunner, BatchRunnerMP
import pathos
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import clear_output
import time
import multiprocessing as mp
import analysis



In [ ]:
# We define our variables and bounds

fixed_params = {
    "network_type": 2
}

problem = {
    'num_vars':6,
    'names': ['similarity_treshold', 'social_influence', 'swingers', 'no_of_neighbors', 'echo_limit',"opinions"],
    'bounds': [[0.01,0.1], [0.01, 0.1], [1, 5], [2,6], [0.80,0.95], [2,5]]
}

# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 30
max_steps = 100
distinct_samples = 10

# We get all our samples here
param_values = saltelli.sample(problem, distinct_samples, calc_second_order= False)


In [ ]:
# Set the outputs
model_reporters = {
                "radical_opinions": compute_radical_opinions,
                "Silent_Spiral": compute_silent_spiral,
                "majority_percentage": compute_majority_opinions,
                "community_no": community_no,
                "transitivity": compute_transitivity,
                "echo_chamber": compute_echo_chamber,
                "echo_size": echochamber_size,
                "echo_count": echochamber_count,          
}

# BatchRunner
batch = BatchRunnerMP(Network, 
                    max_steps=max_steps,
                    fixed_parameters=fixed_params,
                    variable_parameters={name:[] for name in problem['names']},
                    model_reporters=model_reporters, nr_processes=8)
times = []
count = 0
for i in range(replicates):
    for vals in param_values:
        start = time.time()

        # Change parameters that should be integers
        vals = list(vals)

        # Transform to dict with parameter names and their values
        variable_parameters = {}
        for name, val in zip(problem['names'], vals):
            variable_parameters[name] = val
        variable_parameters['network_type']=2
        variable_parameters['malicious_N']=0
        variable_parameters['N']=1000
        variable_parameters['beta_component']=1
        variable_parameters['swingers']=int(variable_parameters['swingers'])
        variable_parameters['no_of_neighbors']=int(variable_parameters['no_of_neighbors'])
        variable_parameters['all_majority']=False
        variable_parameters['opinions']=int(variable_parameters['opinions'])

        batch.run_iteration(variable_parameters, tuple(vals), count)
        count += 1

        clear_output()
        print(f'{count / (len(param_values) * (replicates)) * 100:.2f}% done')
        times.append(time.time() - start)
        meantime = np.mean(times)
        print('Average duration per iteration: %s seconds.'%str(meantime))
        nectime = ((len(param_values) * (replicates))-count)*meantime
        print('Expected time till finish: ', int(nectime/3600), 'hours and ', int((nectime%3600)/60), ' minutes.')
        
        
    
data = batch.get_model_vars_dataframe()
data.to_pickle('./Data/Dirk_Sobol_BA.pkl')


In [ ]:
import _pickle as cPickle
import pandas as pd

print(np.__version__)
print(pd.__version__)

path_pickle = './Data/Dirk_Sobol_BA.pkl'

with open(path_pickle, 'rb') as fo:
    data = cPickle.load(fo, encoding='latin1')

In [ ]:
def plot_index(s, params, i, title=''):
    """
    Creates a plot for Sobol sensitivity analysis that shows the contributions
    of each parameter to the global sensitivity.

    Args:
        s (dict): dictionary {'S#': dict, 'S#_conf': dict} of dicts that hold
            the values for a set of parameters
        params (list): the parameters taken from s
        i (str): string that indicates what order the sensitivity is.
        title (str): title for the plot
    """

    if i == '2':
        p = len(params)
        params = list(combinations(params, 2))
        indices = s['S' + i].reshape((p ** 2))
        indices = indices[~np.isnan(indices)]
        errors = s['S' + i + '_conf'].reshape((p ** 2))
        errors = errors[~np.isnan(errors)]
    else:
        indices = s['S' + i]
        errors = s['S' + i + '_conf']
        plt.figure()

    l = len(indices)
    
    plt.figure(dpi=100)
    plt.title(title)
    plt.ylim([-0.2, len(indices) - 1 + 0.2])
    plt.yticks(range(l), params)
    plt.errorbar(indices, range(l), xerr=errors, linestyle='None', marker='o')
    plt.axvline(0, c='k')


In [ ]:
echo_count = sobol.analyze(problem, data['echo_count'].as_matrix(), print_to_console=True, calc_second_order=False)      
Silent_Spiral = sobol.analyze(problem, data['Silent_Spiral'].as_matrix(), print_to_console=True, calc_second_order=False)      
community_no = sobol.analyze(problem, data['community_no'].as_matrix(), print_to_console=True, calc_second_order=False)      
echo_chamber = sobol.analyze(problem, data['echo_chamber'].as_matrix(), print_to_console=True, calc_second_order=False)      
majority_percentage = sobol.analyze(problem, data['majority_percentage'].as_matrix(), print_to_console=True, calc_second_order=False)      
radical_opinions = sobol.analyze(problem, data['radical_opinions'].as_matrix(), print_to_console=True, calc_second_order=False)      
transitivity = sobol.analyze(problem, data['transitivity'].as_matrix(), print_to_console=True, calc_second_order=False)      


to_plot = [majority_percentage,echo_count,Silent_Spiral,community_no,echo_chamber,radical_opinions,transitivity]
names = ["majority_percentage","echo_count","Silent_Spiral","community_no","echo_chamber","radical_opinions","transitivity"]

In [ ]:
for Si, name in zip(to_plot, names):
    # First order
    plot_index(Si, problem['names'], '1', 'First order sensitivity')
    plt.title(name+'_FO')
    plt.savefig('./img/SOBOL_FirstOrder_'+name+'.png', bbox_inches='tight')   
    plt.show()

    # Total order
    plot_index(Si, problem['names'], 'T', 'Total order sensitivity')
    plt.title('TO_'+name)
    plt.tight_layout()
    plt.savefig('./img/SOBOL_TotalOrder_'+name+'.png',  bbox_inches='tight')    
    plt.show()
    


# OFAT SA

In [ ]:
#### %matplotlib inline
from model import *
from agents import *
from utility import *
from SALib.sample import saltelli
from mesa.batchrunner import BatchRunner, BatchRunnerMP
import pathos
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import clear_output
import time
import multiprocessing as mp
import analysis


fixed_params = {
    "N": 1000,
    "network_type": 2,
    "no_of_neighbors": 3,
    "beta_component": 0.15,
    "social_influence": 0.06,
    "similarity_treshold": .16,
    "swingers": 3,
    "malicious_N":0,
    "echo_limit": .95,
    "all_majority": False,
    "opinions":2,
}
 
problem = {
    'num_vars': 6,
    'names': ['no_of_neighbors', 'similarity_treshold', 'social_influence','swingers','echo_limit','opinions'],
    'bounds': [[2, 6], [0.01, 0.1], [0.01, .1],[1,5],[.05,.2],[2,5]]
}

# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 30
max_steps = 100
distinct_samples = 10


# Set the outputs
model_reporters = {
#                 "preferences": compute_preferences,
#                 "opinions": compute_opinions,
#                 "preference_A": compute_preference_A,
#                 "preference_B": compute_preference_B,
                "radical_opinions": compute_radical_opinions,
                "Silent_Spiral": compute_silent_spiral,
                "majority_percentage": compute_majority_opinions,
                "community_no": community_no,
                "transitivity": compute_transitivity,
                "echo_chamber": compute_echo_chamber,
                "echo_size": echochamber_size,
                "echo_count": echochamber_count,  
}




In [ ]:
data = {}

for i, var in enumerate(problem['names']):
    # Get the bounds for this variable and get <distinct_samples> samples within this space (uniform)
    samples = np.linspace(*problem['bounds'][i], num=distinct_samples)
    
    # Keep in mind that wolf_gain_from_food should be integers. You will have to change
    # your code to acommidate for this or sample in such a way that you only get integers.
    if var == 'no_of_neighbors' or var == 'swingers' or var == "opinions" or var=='N' or var=='network_type' or var=='malicious_N':
        samples = np.linspace(*problem['bounds'][i], num=distinct_samples, dtype=int)
#     print({var: samples})
    fixed_params = {
        "N": 1000,
        "network_type": 2,
        "no_of_neighbors": 3,
        "beta_component": 0.15,
        "social_influence": 0.06,
        "similarity_treshold": .16,
        "swingers": 3,
        "malicious_N":0,
        "echo_limit": .95,
        "all_majority": False,
        "opinions":2,
    }
    

    del fixed_params[var]
#     print(fixed_params)
    batch = BatchRunnerMP(Network, 
                        max_steps=max_steps,
                        iterations=replicates,
                        fixed_parameters=fixed_params,
                        variable_parameters={var: samples},
                        model_reporters=model_reporters,
                        display_progress=True)
    
    batch.run_all()
    
    data[var] = batch.get_model_vars_dataframe()
    

In [ ]:
for i in data.keys():
    data[i].to_pickle('./Data/OFAT_'+i+'.pkl')

Parameter S1 S1_conf ST ST_conf
similarity_treshold 0.330884 0.188688 0.550137 0.159244
social_influence 0.135771 0.148032 0.329089 0.084212
swingers 0.055872 0.149970 0.264796 0.088634
no_of_neighbors 0.006328 0.193664 0.397989 0.194793
echo_threshold -0.086975 0.147927 0.356732 0.144617
Parameter S1 S1_conf ST ST_conf
similarity_treshold 0.011505 0.007873 1.006711 0.688908
social_influence 0.005753 61833321753175953438440161280.000000 1.510067 5410415653402896387658397777920.000000
swingers 0.005753 52754807617912073471597215744.000000 1.510067 4616045666567308201177500352512.000000
no_of_neighbors 0.005753 71135641963862636736699957248.000000 1.510067 6224368671837981730409990324224.000000
echo_threshold 0.005753 52754807617912073471597215744.000000 1.510067 4616045666567307638227546931200.000000
Parameter S1 S1_conf ST ST_conf
similarity_treshold -0.075855 0.208543 1.071320 0.345606
social_influence -0.041573 0.196823 0.863609 0.202496
swingers -0.066955 0.210374 0.907498 0.205424


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [ ]:
import matplotlib.pyplot as plt

reporters = model_reporters.keys()
params = problem['names']

for param in params:
    subdat = pd.read_pickle('./Data/OFAT_'+param+'.pkl')
    print(subdat)
    x = subdat[param]
    all_x = x.unique()
    for reporter in reporters:
        if reporter == 'echo_chamber':
            break
        else:
            y = []
            stds = []
            y_min = []
            y_max = []
            for i in all_x:
                reporter_data = subdat[reporter].loc[subdat[param] == i]
                mean = np.mean(reporter_data)
                std = np.std(reporter_data)
                y.append(mean)
                stds.append(std)
                y_min.append(mean - std) 
                y_max.append(mean + std) 
            plt.plot(all_x, y, c='k')
            plt.fill_between(all_x, y_min, y_max, color = 'steelblue', alpha=.2)
            plt.title('OFAT '+reporter)
            plt.xlabel(param)
            plt.ylabel(reporter)
            plt.tight_layout()
            plt.savefig('./img/OFAT_'+param +'_'+reporter+'.png', bbox_inches='tight')            
            plt.show()


In [ ]:
def plot_param_var_conf(ax, df, var, param, i):
    """
    Helper function for plot_all_vars. Plots the individual parameter vs
    variables passed.

    Args:
        ax: the axis to plot to
        df: dataframe that holds the data to be plotted
        var: variables to be taken from the dataframe
        param: which output variable to plot
    """
    x = df.groupby(var).mean().reset_index()[var]
    y = df.groupby(var).mean()[param]

    replicates = df.groupby(var)[param].count()
    err = (1.96 * df.groupby(var)[param].std()) / np.sqrt(replicates)

    ax.plot(x, y, c='k')
    ax.fill_between(x, y - err, y + err)

    ax.set_xlabel(var)
    ax.set_ylabel(param)

def plot_all_vars(df, param):
    """
    Plots the parameters passed vs each of the output variables.

    Args:
        df: dataframe that holds all data
        param: the parameter to be plotted
    """

    f, axs = plt.subplots(3, figsize=(7, 10))
    
    for i, var in enumerate(problem['names']):
        plot_param_var_conf(axs[i], data[var], var, param, i)

for param in variable_params.keys():
    plot_all_vars(data[param], param)
    plt.show()